In [ ]:
from pianoq.misc.mplc_writeup_imports import *
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_10_30_make_mirror_work'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

# WFM 

In [16]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1

for dz in [4, 6, 10, 12]:
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    masks_path = fr'{dir_path}\{timestamp}_U1U_dead_zone_450pix_dz10_{dz}mm.masks'
    mplc_res_path = fr'{dir_path}\{timestamp}_U1U_dead_zone_450pix_dz10_{dz}mm.mplcsim'
    mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=450e-6,
                                        last_plane_extra_dist=dz*1e-3)
    mplc_sim.res._calc_fidelity()
    print(dz)
    print(mplc_sim.res.fidelity)

100%|██████████| 15/15 [03:20<00:00, 13.37s/it]


In [7]:
mplc_sim.res.saveto(mplc_res_path, smaller=True)
mplc_sim.res.show_overlap()
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9885901232826865

In [9]:
mplc_sim.res.show_all(2)

# Hardware

In [2]:
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=35)
integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

God timetagger


In [3]:
backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)


Got Zaber motors
Got Thorlabs motors


# Single counts

In [ ]:
for dz in [4, 6, 10, 12]:
    
    # temp phases, that are probably better than nothing 
    phases_path = rf'{dir_path}\2024_10_31_10_15_02_fine_new_mirror_from_zero_more.phases'
    # masks_path = fr'{dir_path}\2024_10_30_13_10_41_U1U_dead_zone_450pix_dz10_8mm.masks'
    masks_path = glob.glob(rf'{dir_path}\*dz10_{dz}mm.masks')[0]
    
    masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    time.sleep(0.5)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_5spots_fine_signal_dead_zone_450um_dz10_{dz}mm_p10_tilts_35.scan'
    
    start_x = 12.75
    end_x = 13.35
    start_y = 4.75
    end_y = 6.6
    
    resolution = 1
    x_pixels = 24 // resolution
    y_pixels = 76 // resolution
    pixel_size_x = 0.025 * resolution
    pixel_size_y = 0.025 * resolution
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path) 
    tt = get_timetagger(integration_time=1)  # important to have a new tt 
    t1 = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)
    
    time.sleep(2)
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_5spots_fine_idler_dead_zone_450um_dz10_{dz}mm_p10_tilts_35.scan'
    
    start_x = 9.25
    end_x = 9.75
    start_y = 3.6
    end_y = 5.35
    
    x_pixels = 20 // resolution
    y_pixels = 70 // resolution
    pixel_size_x = 0.025 * resolution
    pixel_size_y = 0.025 * resolution
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)
    tt2 = get_timetagger(integration_time=1)
    t2 = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt2)
    t1.join()
    send_telegram_message(f'Done one singles scan {dz=}')
    t2.join()    
    send_telegram_message(f'Done second singles scan {dz=}')

# mark spots 

In [9]:
locs_signal_path = fr'{dir_path}\{tnow()}_s2_sig.locs'
locs_idler_path = fr'{dir_path}\{tnow()}_s1_idl.locs'

res_idl = ScanResult(fr'{dir_path}\2024_10_30_14_03_30_5spots_fine_idler_dead_zone_450um_dz10_8mm_p10_tilts_35.scan')
res_sig = ScanResult(fr'{dir_path}\2024_10_30_14_03_28_5spots_fine_signal_dead_zone_450um_dz10_8mm_p10_tilts_35.scan')
# res_idl.show_singles()
# res_sig.show_singles()
res_idl.get_xys(1, saveto_path=locs_idler_path)
res_sig.get_xys(2, saveto_path=locs_signal_path)

[(13.025198967698966, 5.0285422910422914),
 (13.042914585414582, 5.372731435231436),
 (13.047976190476188, 5.709328171828172),
 (13.093530636030634, 6.050986513486514),
 (13.123900266400264, 6.379990842490844)]

# Phases 

In [14]:
masks_path = fr'{dir_path}\2024_10_30_13_10_41_U1U_dead_zone_450pix_dz10_8mm.masks'
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=None)  # have this with clean phases when sending to the phase finder 
mplc.load_masks(masks)

saveto_path = fr'{dir_path}\{tnow()}_fine_new_mirror_from_zero_more.phases'
integration_time = 50

# previous phases result 
old_phases_path = fr"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"
old_phases_result = PhaseFinderResult(old_phases_path)
old_phases_result.reload()
initial_phases = old_phases_result.phases


pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=saveto_path,
                 modes_to_keep=modes_to_keep, intial_phases=initial_phases, coin_window=tt.coin_window, no_hw_mode=True)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_idl = np.load(fr'{dir_path}\2024_10_30_15_19_40_s1_idl.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2024_10_30_15_19_40_s2_sig.locs')['locs']

# i,j of corr matrix that is supposed to be strong 
i = 1
j = 1
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])
mplc.restore_location()
time.sleep(1)
pf.find_phases()

# Correlations 

In [15]:
backlash = 0.0
wait_after_move = 0.3
integration_time = 50
coin_window = 1e-9

phases_path = rf'{dir_path}\2024_10_31_10_15_02_fine_new_mirror_from_zero_more.phases'
masks_path = fr'{dir_path}\2024_10_30_13_10_41_U1U_dead_zone_450pix_dz10_8mm.masks'
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)

scan_path = fr'{dir_path}\{tnow()}.dscan'

locs_idl = np.load(fr'{dir_path}\2024_10_30_15_19_40_s1_idl.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2024_10_30_15_19_40_s2_sig.locs')['locs']

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)

dps.m_sig_x, dps.m_sig_y, dps.m_idl_x, dps.m_idl_y, dps.time_tagger = mxs, mys, mxi, myi, tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)
dps.scan()